<a href="https://colab.research.google.com/github/jeongwoo22/ESAA_assignments/blob/main/OB_%EA%B3%BC%EC%A0%9C7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 9. 비지도학습

- 대부분의 머신러닝 애플리케이션 ==> 지도 학습 기반 BUT 사용할 수 있는 데이터는 대부분 레이블이 없음. 즉 특성 x는 있지만 레이블 Y 가 없다는 것
- EX) 제조 제품의 결함을 판단하기 위해서는 '결함', '정상' 이라고 레이블을 부여해야 함. ==> 사람이 모든 사진에 레이블을 붙일 필요 없이 알고리즘이 레이블이 없는 데이터를 바로 사용하면 좋다 = 비지도 학습

- 비지도 학습 방법 종류 : 차원축소, 군집, 이상치 탐지, 밀도 추정
- 군집 : 비슷한 샘플을 클러스터로 모음 ==> 데이터 분석, 고객 분류, 추천 시스템, 검색 엔진, 이미지 분할, 준지도 학습 등에 사용 가능 
- 이상치 탐지 : '정상' 데이터가 어떻게 보이는지 학습 후 비정상 샘플을 감지 하는데 사용
- 밀도 추정 : 데이터셋 생성 확률 과정의 PDF 추정. 밀도 추정은 이상치 탐지에 많이 사용됨. 밀도가 낮은 영역에 놓인 샘플이 이상치일 가능성 높음

-----

## 9.1 군집

- 군집 EX ==> 식물학자는 아니지만, 꽃을 보고 비슷해 보이는 꽃을 모으는 것
- 클러스터 : 비슷한 샘플을 구별해 하나의 클러스터 OR 비슷한 샘플의 그룹으로 할당하는 작업
- 각 샘플은 하나의 그룹에 할당 ==> BUT 분류가 아니라 군집은 비지도 학습
- 이유 : 분류는 레이블이 되어 있음. 즉 클래스인 품종이 구분되어 샘플에 나타나 있음. BUT 군집에서는 동일한 데이터 셋이지만 레이블이 없다

- 붓꽃 예시 : 2개 특성 기준으로 그래프를 만들어 보았을 때, 사람 눈으로 구분할 수 있는 클러스터 외에, 다른 클러스터들은 다른 모든 특성들을 더 사용하여 구분할 수 있다. 
- 예로, 가우시안 혼합 모델을 사용하면 150개 중 5개 샘플만 잘못된 클러스터로 할당 

- 군집 사용 예시 : 고객분류, 데이터 분석, 차원 축소 기법, 이상치 탐지, 준지도 학습, 검색 엔진, 이미지 분할
- 고객 분류 : 고객을 구매 이력, 웹사이트 행동 으로 클러스터를 모음. 고객 그룹을 만들어 제품 추천, 마케팅 전략, 추천 시스템 등을 적용
- 데이터 분석 : 새로운 데이터 셋을 분석할 때
- 차원 축소 기법 : 한 데이터 셋에 군집 알고리즘을 적용해 각 클러스터에 대한 샘플 친화성을 측정. 친화성이란, 샘플이 클러스터에 얼마나 잘 맞는지를 측정하는 것. K 개의 클러스터가 있으면 K 차원이 됨. 
- 이상치 탐지 : 모든 클러스터에 친화성이 낮은 샘플은 이상치. 이상치 탐지는 제조 분야에서 결함을 감지할 때 유용. 혹은 부정 거래 감지에 활용
- 준지도 학습 : 레이블 된 샘플이 적을 때, 군집을 수행하고 동일한 클러스터에 있는 모든 샘플에 레이블 전파 가능. 
- 검색 엔진 : 제시된 이미지와 비슷한 이미지를 찾음. 먼저 데이터 베이스에 있는 모든 이미지에 군집 알고리즘 적용. 비슷한 이미지는 동일한 클러스터에 속함. 
- 이미지 분할 : 색을 기반으로 픽셀을 클러스터로 모음. 그 뒤 픽셀의 색을 해당 클러스터의 평균 색으로 바꿈. 이는 이미지의 색상 종류를 크게 줄임. 

- 클러스터 : 보편적 정의 없음 
- 센트로이드 : 특정 포인트를 중심으로 모인 샘플 찾음

---------

### 9.1.1 k - 평균

- K 평균 : 반복 몇 번으로 여러 종류의 데이터 셋을 빠르고 효율적으로 클러스터로 묶음 
- K 평균 알고리즘 : 각 클러스터의 중심을 찾고 가장 가까운 클러스터에 샘플 할당

In [9]:
import numpy as np
import os
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

- 1단계 : 먼저 알고리즘이 찾을 클러스터 개수 k 를 지정
- ==> k=5 라고 하면, 각 샘플은 5개의 클러스터 중 하나에 할당 
- 군집에서 레이블 : 알고리즘이 샘플에 할당한 클러스터의 인덱스 (분류에서의 클래스 레이블과 다름) 

In [11]:
# kmeans.cluster_centers_     ====>     를 통해 알고리즘이 찾은 센트로이드 5개도 확인 가능

# 새로운 샘플 x_new 를 주었을 때, 가장 가까운 센트로이드 클러스터를 할당할 수 있다. 
# X_new = np.array([[0,2], [3,2], [-3,3], [-3, 2.5]])

- k 평균 알고리즘 단점 : 클러스터의 크기가 많이 다르면 잘 작동하지 않음. 왜냐하면 샘플을 클러스터에 할당할 때 센트로이드까지 거리를 고려하는 것이 전부이기 때문

- 하드 군집(hard clustering) : 샘플을 하나의 클러스터에 할당하는 것
- 소프트 군집(soft clustering) : 샘플에 클러스터마다 점수를 부여하는 것. 이 때의 점수는 샘플과 센트로이드 사이의 거리 or 가우시안 유사도 점수 (친화성 점수) 등 가능 

In [12]:
# kmeans.transform(X_new)     ====>   X_new 의 4개 샘플이 총 5개의 센트로이드에거 각각 얼마나 떨어져 있는지 반환함. 즉 4행 5열 array 반환

- 고차원 데이터 셋을 위의 방식으로 반환하면 k- 차원 데이터 셋이 만들어짐 ==> 효율적인 비선형 차원 축소 기법 가능

#### k- 평균 알고리즘

- 알고리즘 작동 방법 설명

- KMeans 클래스 : 기본적으로 최적화된 알고리즘 적용, 매개변수는 init = "random", n_init = 1, algorithm = "full" 로 설정

- 레이블이나 센트로이드가 주어지지 않으면 ? ===> 처음에는 센트로이드를 랜덤하게 선정 (무작위로 k 개의 샘플을 뽑아 그 위치를 센트로이드로 정함) 
- 다음으로 ==> 샘플에 레이블을 할당하고 센트로이드를 업데이트하고, 샘플에 레이블을 할당하고 센트로이드를 업데이트 하면서 센트로이드에 변화가 없을 때까지 계속함 

- k- 평균 알고리즘의 단점 : 솔루션이 초기 센트로이드 배정에 달려 있으므로, 센트로이드 초기화에 따라서 솔루션이 달라짐. 따라서 운이 없을 때 최적이 아닌 솔루션에 도달할 위험 존재

#### 센트로이드 초기화 방법

- 센트로이드 초기화의 위험 개선방법

- 방법 1 : 처음에 센트로이드 위치를 근사하게나마 알기 (또 다른 군집 알고리즘을 실행해서) ==> 센트로이드 리스트 배열을 넣어 실행 
- 방법 2 : 랜덤 초기화를 다르게 하여 알고리즘을 여러번 실행하고 가장 좋은 솔루션 선택. n_init ===> 랜덤 초기화 횟수 조절 (기본 값은 10) 
- 최종 솔루션 알 수 있는 방법 ===> 이니셔 라는 성능 지표 사용!

- 모델의 이니셔(inertia) : 각 샘플과 가장 가까운 센트로이드 사이의 평균 제곱 거리
- KMeans 클래스는 알고리즘은 n_init 번 실행하여 이니셔가 가장 낮은 모델 반환. ====> 랜덤 초기화가 n_init 번 연속으로 운이 나쁘지 않다면 좋은 모델이 선택됨. 

In [ ]:
# kmeans.inertia_    ====> 모델의 이니셔 확인

# kmeans.score(X)    =====> 이니셔의 음숫값 반환. 왜냐하면 score() 매서드는 큰 값이 좋은 것이란 규칙을 따라야 하기 때문. 

- k 평균 ++ 알고리즘 : 다른 센트로이드와 거리가 먼 센트로이드를 선택하는 초기화 단계
- KMeans 클래스는 기본적으로 이 초기화 방법 사용. 원래 방식을 사용하고 싶으면 =====> init 매개변수를 "random" 으로 지정하면 됨. 

#### k - 평균 속도 개선과 미니배치 k - 평균

- 찰스 엘칸 : k - 평균 알고리즘에서 불 필요한 거리 계산을 많이 피함으로서 알고리즘의 속도 높이기 ==> 삼각 부등식 사용
- 이걸 다시 KMeans 클래스에서 기본으로 사용 ==> 원래 알고리즘을 사용하려면 algorithm 매개변수를 "full" 로 지정

- 데이비드 스컬리 : 미니 배치를 사용해 센트로이드를 이동하는 알고리즘 개발 ===> 일반적인 알고리즘 속도를 3배에서 4배 정도로 높임. 또한 메모리에 들어가지 않는 대량의 데이터 셋에 군집 알고리즘을 적용할 수 있음. 

In [16]:
# 사이킷런이 MiniBatchMeans 클래스에 이 알고리즘을 구현. 

# from sklearn.cluster import MiniBatchMeans
# minibatch_kmeans = MiniBatchMeans(n_clusters = 5)

- 데이터셋이 메모리에 들어가지 않으면, ==> memmap 클래스를 사용하는 것. 또는 미니배치 k-means 클래스의 partial_fit() 매서드에 한 번에 하나의 미니배치를 전달할 수 있다. 
- 미니 배치 k_means 알고리즘이 일반 k_means 알고리즘보다 빠르지만, 이니셔는 일반적으로 조금 더 나쁘다. 특히 클러스터의 개수가 증가할 때 그렇다. 

#### 최적의 클러스터 개수 찾기

- 데이터를 볼 때 클러스터 개수가 보이면 몰라도, 보통 일반적으로 클러스터 개수 k 개가 몇개인지, 어떻게 설정해야 할지 알 수 없다. 
- 이니셔로 조절? : 못함. 왜냐하면, k 가 증가함에 따라 이니셔가 같이 작아지므로. 잘못된 k 를 지정해도 이니셔가 작을 수 있기 때문. 

- 엘보 : 이너셔 그래프를 클러스터 개수 k 의 함수로 그렸을 때 그래프가 꺾이는 지점. ==> EX) 엘보가 4라면, 4까지 증가할 때 빠르게 줄어들다가 4 이후부터는 이너셔가 느리게 감소함. 이럴 때 4는 좋은 선택이 됨. ====> BUT 이 방법은 너무 엉성

- 실루엣 점수(Silhouette score) : 앞 보다 더 정확하지만, 계산 비용이 많이 듬. 실루엔 점수는, 모든 샘플에 대한 실루엣 계수의 평균
- 실루엣 계수 : (b-a)/ max(a,b) a는 동일한 클러스터에 있는 다른 샘플까지의 평균 거리. (클러스터 내부의 평균 거리) b 는 가장 가까운 클러스터까지 평균 거리 
- 실루엣 계수는 -1~1까지 변동. 
- 실루엣 계수 1에 가까움 : 자신의 클러스터 안에 잘 속해있고, 다른 클러스터와 멀리 떨어져 있음
- 실루엣 계수가 0에 가까움 : 클러스터 경계에 위치 
- 실루엣 계수 -1에 가까움 : 샘플이 잘못된 클러스터에 할당

In [17]:
# 실루엣 점수 계산 

# from sklearn.metrics import silhouette_score
# silhouette_score(X, kmeans.labels_)

- 실루엣 계수를 비교해보면, 이너셔만 계산했을 때보다 훨씬 많은 정보를 준다는 것을 알 수 이따. 

- 실루엣 다이어그램 : 모든 샘플의 실루엣 계수를 할당된 클러스터와 계숫값으로 정렬하여 더 많은 정보를 얻을 수 있음 
- 클러스터마다 칼 모양의 그래프가 그려짐
- 높이 : 클러스터가 포함하고 있는 샘플의 개수
- 너비 : 클러스터에 포함된 샘플의 정렬된 실루엣 계수 ===> 넓을 수롣 좋음

------------

### 9.1.2 k- 평균의 한계

- k 평균 장점 : 속도 빠름. 확장 용의 
- 단점 1 : 최적이 아닌 솔루션을 피하려면 알고리즘을 여러번 실행해야 함. 
- 단점 2 : 클러스터 개수를 지정해야함. 
- 단점 3 : 클러스터의 크기나 밀집도가 서로 다르거나 원형이 아닌 경우 잘 작동하지 않음. 

- 데이터에 따라 잘 수행할 수 있는 군집 알고리즘이 다르다. ===> ex) 가우시안 혼합 모델은 타원형 클러스터에 잘 작동
- +++ k-평균을 하기 전에 특성의 스케일을 맞추는 것이 중요. 그렇지 않으면 클러스터가 길쭉해지고 k-평균의 결과가 좋지 않음. 

-------

### 9.1.3 군집을 사용한 이미지 분할

- 이미지 분할(image segmentation) : 이미지를 세그멘트 여러개로 분할하는 작업
- 시멘틱 분학 (semantic segmentation) : 동일한 종류의 물체에 속한 모든 픽셀을 같은 세그먼트에 할당 
- ex) 자율 주행 자동차 비전 시스템에서, 보행자 이미지를 구성하는 모든 픽셀은 '보행자' 세그먼트에 할당. 각 보행자는 다른 세그먼트가 됨. 
- 시맨틱 혹은 인스턴스 분할에서 최고 수준의 성능을 내려면 합성곱 신경망을 사용한 복잡한 모델을 사용해야 함. 

- 색상 분할(color segmentation) : 동일한 색상을 가진 픽셀을 같은 세그멘트에 할당
- ex) 인공위성 사진을 분석하여 한 지역의 전체 산림 면적이 얼마나 되는지 측정 

In [19]:
# from matpltilb.image import imread
# image = imread(os.path.join("images", "unsupervised_learning", "laybug.png"))
# image.shape # 필셀  행, 열 개수, 컬러 채널 개수 --> rgb 

- rgb : 각 픽셀에 대한 빨강, 초록, 파랑의 강도를 담은 3d 벡터 ==> 0.0 ~ 1.0 사이 , imageio.imread() 를 사용하면 0~255 사이 
- 어떤 이미지는 더 적은 채널을 가짐 : 흑백 (채널이 1개) 
- 어떤 이미지는 더 많은 채널을 가짐 : 투명도를 위해 알파 채널 (alpha channel)을 가진 이미지 or 위성 이미지 등
- 위성 이미지 : 여러 전자기파에 대한 채널을 포함(ex- 적외선) 

In [20]:
# # rgb 색상의 리스트를 색상 별 클러스터로 모음

# X= image.reshape(-1,3)
# kmeans = KMeans(n_clusters = 8).fit(X)
# segmentation_img = kmeans.cluster_centers_[kmeans.labels_]
# segmentation_img = segmented_img.reshape(image.shape)

- 예를 들어 모든 초록색 : 하나의 컬러 클러스터로 만듦. ==> 컬러 클러스터의 평균 컬러를 찾음 ex) 밝은 초록

- 예제 그림에서 클러스터 개수를 8개 보다 작게 하면, 무당벌레의 화려한 빨강색이 독자적인 클러스터를 만들지 못하고 주위 색에 합쳐짐. ===> 색이 화려해도 다른 부분에 비해서 크기가 작기 때문에 

--------------

### 9.1.4 군집을 사용한 전처리

- 군집은 차원축소에 효과적. 특히 지도 학습 알고리즘을 적용하기 전에 전처리 단계에서 사용하기 좋음. 

- 차원 축소에 군집 사용 예를 위해 ==> 숫자 데이터 셋 예제 사용

In [21]:
from sklearn.datasets import load_digits

X_digits, y_digits = load_digits(return_X_y=True)

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits)

In [23]:
# 로지스틱 회귀 모델 훈련 
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [24]:
# 테스트 세트에서 정확도 평가 
log_reg.score(X_test, y_test)

0.9511111111111111

- k-means 를 전처리 단계로 사용해 정확도가 더 좋아지는지 확인해보기 

In [25]:
# 파이프 라인을 만들어 먼저 훈련 세트를 ===> 50개의 클러스터로 모으기 
from sklearn.pipeline import Pipeline

pipeline = Pipeline([("kmeans", KMeans(n_clusters = 50)), 
                     ("log_reg", LogisticRegression())
                     ])
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                ('log_reg', LogisticRegression())])

- 참고 : 숫자가 10개 이므로 클러스터 개수를 10개로 지정할 수 있으나, 숫자를 쓴 방식이 저마다 다르므로 처음에 50처럼 클러스터 개수를 더 크게 하는 것이 좋다

------------

In [26]:
# 분류 파이프라인 평가 
pipeline.score(X_test, y_test)

0.9733333333333334

- 오차율을 30% 까지 줄임

- BUT 클러스터 개수 k 를 임의로 정하지 않으면 더 성능이 좋을 수 있다 ===> k_평균이 분류 파이프 라인의 하나의 전처리 단계이기 때문에, 이전처럼 실루엣 분석을 하거나 이너셔가 감소되는걸 확인할 필요 없이 좋은 k 값을 찾는게 더 쉬움. 
- 가장 좋은 k 값은 교차 검증에서 가장 좋은 분류 성능을 내는 값 

In [30]:
# # gridsearchcv 로 최적의 클러스터 개수 찾아보기 
# from sklearn.model_selection import GridSearchCV

# param_grid = dict(kmeans__n_clusters = range(2,100))
# grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose = 2)
# grid_clf.fit(X_train, y_train)


In [31]:
# grid_clf.best_params_
# grid_clf.score(X_test, y_test)

- 최선의 k 값으로 모델을 만들었을 때 ==> k=99 개를 햇을 때 인데, 정확도가 크게 향상된 것을 볼 수 있다. 범위에서 99가 최대였으니, 99보다 더 큰 값을 선택해 탐색해볼 만함

### 9.1.5 군집을 사용한 준지도 학습

- 준지도 학습 : 군집을 사용하는 또 다른 사례 ==> 레이블이 없는 데이터가 많고, 레이블이 있는 데이터는 적을 때 사용 

In [32]:
# 숫자 데이터 셋에서 레이블된 50개 샘플에 로지스틱 회귀 모델 훈련
n_labeled = 50
log_reg = LogisticRegression()
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [33]:
log_reg.score(X_test, y_test)

0.84

- 정확도 개선 방법 : 먼저 훈련 세트 50개 클러스터로 모으기. 다음으로 각 클러스터에서 대표 이미지(representative image)
- 대표이미지 : 센트로이드에 가장 가까운 이미지

In [35]:
# 대표 이미지 찾기 
k= 50
kmeans = KMeans(n_clusters=k, random_state=42)
X_digits_dist = kmeans.fit_transform(X_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]

- 수동으로 레이블 할당

- 레이블된 50개 샘플로 이루어진 데이터 셋으로 모델 훈련을 해보면 훨씬 성능이 증가한 것을 볼 수있음. 
- 샘플에 레이블 부여 : 비용이 많이 듦 =====> 무작위 샘플 대신 대표 샘플에 레이블을 할당하는 것이 좋음 

- 레이블 전파 (label propagation) : 레이블을 동일한 클러스터에 있는 모든 샘플에게 전파하는 것
- 클러스터 경계에 있는 샘플에 잘못된 레이블이 부여 되었을 수 있으니, 센트로이드와 가까운 샘플 20% 에만 가까운 레이블을 전파해보기

- 능동학습 (active learning) : 전문가가 학습 알고리즘과 상호작용하여 알고리즘이 요청할 때 특정 샘플의 레이블 제공
- 불확실성 샘플링 (uncertainty sampling) : 능동학습의 전략 중 하나 

-------------

### 9.1.6 DBSCAN

- descan : 국부적인 밀집도를 잘 측정하여 임의의 모양을 가진 클러스터를 식별함. 
- descan 작동 방식은 교재 참고 
- descan 은 모든 클러스터가 충분히 밀집되어 있고, 밀집되지 않은 지역과 잘 구분될 때 성능이 좋음 


In [37]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X,y  = make_moons(n_samples = 1000, noise =0.05)
dbscan = DBSCAN(eps = 0.05, min_samples = 5)
dbscan.fit(X)

DBSCAN(eps=0.05)

- labels : 샘플의 레이블 저장
- core_sample_indices_ : 핵심 샘플의 인덱스 
- components_ : 핵심 샘플 자체 

- DBSCAN 클래스는 predict() 를 제공안하고, fit_predict() 를 제공 =====> 새로운 샘플에 대해 클러스터 예측 못함

- DBSCAN 장점 : 간단하지만 강력함
- 클러스터의 모양과 개수에 상관없이 감지 가능
- 이상치에 안정적ㅇ
- 하이퍼 파라미터 2개 : eps, min_samples
- 단점 : 클러스터 간의 밀집도가 다르면 모든 클러스터를 올바르게 잡아내기 어려움

-------------

### 9.1.7 다른 군집 알고리즘

- 병합 군집
- BIRCH
- 평균 - 이동
- 유사도 전파
- 스펙트럼 군집

------------

## 9.2 가우시안 혼합

- 가우시안 혼합 모델(GMM) : 샘플이 파라미터가 알려지지 않은 여러개의 혼합된 가우시안 분포에서 생성되었다고 가정하는 확률 모델
- 하나의 가우시안 분포에서 생성된 모든 샘플은 하나의 클러스터 형성
- 일반적으로 타원형 
- 각 클러스터는 타원의 모양, 크기, 밀집도, 방향이 다름 : 샘플이 주어지면 가우시안 분포 중 하나에서 생성된 것

----

### 9.2.1 가우시안 혼합을 사용한 이상치 탐지

- 이상치 탐지 : 보통과 많이 다른 샘플을 감지하는 작업
- 밀도가 낮은 지역에 있는 모든 샘플이 이상치

In [38]:
# densities = gm.score_samples(X)
# density_threshold = np.percentile(densities, 4)
# anomalies = X[densities < density_threshold]

---------

### 9.2.2 클러스터 개수 선택하기

- BIC, AIC 를 통하여 K 개수 결정함

--------

### 9.2.3 베이즈 가우시안 혼합 모델

- 베이즈 들어가면 MAP(max a posterier)

------

### 9.2.4 이상치 탐지와 특이치 탐지를 위한 다른 알고리즘